In [1]:
# @title ###### Licensed to the Apache Software Foundation (ASF), Version 2.0 (the "License")

# Licensed to the Apache Software Foundation (ASF) under one
# or more contributor license agreements. See the NOTICE file
# distributed with this work for additional information
# regarding copyright ownership. The ASF licenses this file
# to you under the Apache License, Version 2.0 (the
# "License"); you may not use this file except in compliance
# with the License. You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing,
# software distributed under the License is distributed on an
# "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
# KIND, either express or implied. See the License for the
# specific language governing permissions and limitations
# under the License

In [9]:
#installing dependencies
!pip install pandas==1.4.4
!pip install numpy==1.24.4
!pip install apache_beam==2.56.0
!pip install redis==5.0.1
!pip install langchain==0.1.14 #used for chunking


[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: /usr/local/opt/python@3.10/bin/python3.10 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: /usr/local/opt/python@3.10/bin/python3.10 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: /usr/local/opt/python@3.10/bin/python3.10 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: /usr/local/opt/python@3.10/bin/python3.10 -m pip install --upgrade pip


In [1]:
#Imports Required for the notebook
import pandas as pd
import numpy as np
import apache_beam as beam
from apache_beam.ml.transforms.base import MLTransform
from apache_beam.transforms.enrichment import Enrichment
from apache_beam.ml.transforms.embeddings.huggingface import SentenceTransformerEmbeddings
import tempfile
import redis
import redis_connector
import redis_enrichment
from redis_connector import *
from redis_enrichment import *
from redis.commands.search.indexDefinition import (IndexDefinition,IndexType)
from redis.commands.search.query import Query
from redis.commands.search.field import (TextField,VectorField)
from chunks_generation import *

In [2]:
#To check beam version installed 
beam.__version__

'2.56.0'

In [3]:
import json

with open('hf_small_wikipedia.json', 'r') as j:
     contents = json.loads(j.read())

# For now Reading json data locally

In [2]:
import json

with open('hf_small_wikipedia.json', 'r') as j:
     contents = json.loads(j.read())


print(contents[:1])


[{'id': '12', 'url': 'https://en.wikipedia.org/wiki/Anarchism', 'title': 'Anarchism', 'text': 'Anarchism is a political philosophy and movement that is skeptical of all  ... \nSocial theories\nSocialism'}]


In [4]:
type(contents)

list

# Create Redis Client for connecting to Redis Vector Database

In [5]:
REDIS_HOST =  "localhost"
REDIS_PORT = 6379
REDIS_PASSWORD = "" # default for passwordless Redis

# Connect to Redis
redis_client = redis.Redis(
    host=REDIS_HOST,
    port=REDIS_PORT,
    password=REDIS_PASSWORD
)
redis_client.ping()

True

# Creating a Search Index
Below cells will show how to specify and create a search index in Redis vector DB. Below are the following steps:

1) Set some constants for defining our index like the distance metric and the index name
2) Define the index schema with RediSearch fields
3) Create the index

In [6]:
#Constants
EMBEDDING_MODEL = 'all-MiniLM-L6-v2' # Embedding model name to be use with ML Transform
VECTOR_DIM = 384                     # length of the vector for above embedding model
VECTOR_NUMBER = 2                    # initial number of vectors
INDEX_NAME = "embeddings-index"      # name of the search index      
PREFIX = "doc"                       # prefix for the document keys           
DISTANCE_METRIC = "COSINE"           # distance metric for the vectors (ex. COSINE, IP, L2)

In [7]:
# Define RediSearch fields for each of the columns in the dataset
url = TextField(name="url")
title = TextField(name="title")
title_embedding = VectorField("title_vector",
    "FLAT", {
        "TYPE": "FLOAT32",
        "DIM": VECTOR_DIM,
        "DISTANCE_METRIC": DISTANCE_METRIC,
        "INITIAL_CAP": VECTOR_NUMBER,
    }
)

text = TextField(name="text")
text_embedding = VectorField("text_vector",
    "FLAT", {
        "TYPE": "FLOAT32",
        "DIM": VECTOR_DIM,
        "DISTANCE_METRIC": DISTANCE_METRIC,
        "INITIAL_CAP": VECTOR_NUMBER,
    }
)
fields = [url, title, title_embedding, text, text_embedding]


In [8]:
# Check if index exists
try:
    redis_client.ft(INDEX_NAME).info()
    print("Index already exists")
except:
    # Create RediSearch Index
    redis_client.ft(INDEX_NAME).create_index(
        fields = fields,
        definition = IndexDefinition(prefix=[PREFIX], index_type=IndexType.HASH)
)

Index already exists


# Creating Knowledge Base in Redis Vector Database
After creating a search index, we can load documents into it. We will use the same documents we used in the previous cell.

In [9]:
#Insertion Pipeline

artifact_location = tempfile.mkdtemp()
generate_embedding_fn = SentenceTransformerEmbeddings(model_name='all-MiniLM-L6-v2',
                                                               columns=['title','text'])
with beam.Pipeline() as p:
    embeddings = (
        p  
        | "Read data" >> beam.Create(contents) 
        | "Generate text chunks" >> ChunksGeneration(chunk_size = 500, chunk_overlap = 0, chunking_strategy = ChunkingStrategy.SPLIT_BY_TOKENS)
        | "Insert document in Redis" >> InsertDocInRedis(host='127.0.0.1',port=6379, batch_size=10)
        | "Generate Embeddings" >> MLTransform(write_artifact_location=artifact_location).with_transform(generate_embedding_fn) 
        | "Insert Embedding in Redis" >> InsertEmbeddingInRedis(host='127.0.0.1',port=6379, batch_size=10,embedded_columns=['title','text'])
    )

INFO:root:Missing pipeline option (runner). Executing pipeline using the default runner: DirectRunner.


INFO:apache_beam.runners.worker.statecache:Creating state cache with size 104857600
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
2024-08-09 13:01:57.330902: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTr

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:redis_connector:Inserting documents in Redis. Total docs: 10
INFO:redis_connector:Inserting documents complete.
INFO:redis_connector:Inserting documents in Redis. Total docs: 9
INFO:redis_connector:Inserting documents complete.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:root:BatchElements statistics: element_count=349 batch_count=35 next_batch_size=20 timings=[(1, 0.14947915077209473), (2, 0.4767780303955078), (1, 0.1831817626953125), (2, 0.2000288963317871), (1, 0.14716172218322754), (2, 0.19555902481079102), (4, 0.568000078201294), (3, 0.49921202659606934), (1, 0.15181899070739746), (2, 0.2397456169128418), (2, 0.19758391380310059), (3, 0.259962797164917), (6, 0.6725080013275146), (7, 0.7242469787597656), (7, 0.6019473075866699), (7, 0.47342967987060547), (10, 0.9236979484558105), (13, 1.1121737957000732), (14, 1.130352258682251), (12, 0.9904649257659912), (19, 1.336167812347412), (13, 0.8261549472808838), (15, 1.156790018081665), (17, 1.272038221359253), (15, 1.0295050144195557), (14, 0.973175048828125), (16, 1.2297940254211426), (15, 0.8902089595794678), (23, 1.609447956085205), (19, 1.1090152263641357), (22, 1.5956041812896729), (18, 1.3837130069732666), (25, 1.715193748474121), (17, 1.1607699394226074)]


## Pipeline Steps:

Now that we have ingested the documents in Redis, we will create a embeddings transform, which is used for storing the text and its embedding in redis vector db


# Running Search Queries/ Perform Enrichment

## Pipeline Steps:
Create a search transform, which emits the document Id, vector score along with the matching text from knowledge base


In [10]:
#  Enchriment Pipeline 


data = [{'text':'What is Anarchy ?'}]

artifact_location = tempfile.mkdtemp()
generate_embedding_fn = SentenceTransformerEmbeddings(model_name='all-MiniLM-L6-v2',
                                                                 columns=['text'])

redis_handler = RedisEnrichmentHandler(redis_host='127.0.0.1', redis_port=6379)
                                       

with beam.Pipeline() as p:
  _ = (
      p
      | "Create" >> beam.Create(data)
      | "Generate Embedding" >> MLTransform(write_artifact_location=artifact_location).with_transform(generate_embedding_fn)
      | "Enrich W/ Redis" >> Enrichment(redis_handler)
      | "Print" >> beam.Map(print)
  )

INFO:root:Missing pipeline option (runner). Executing pipeline using the default runner: DirectRunner.
INFO:apache_beam.runners.worker.statecache:Creating state cache with size 104857600
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:root:BatchElements statistics: element_count=1 batch_count=1 next_batch_size=1 timings=[]


Row(text=[0.004666077438741922, 0.05869913473725319, -0.07399024069309235, 0.022393187507987022, 0.039686284959316254, -0.034507881850004196, 0.064857617020607, -0.047807302325963974, -0.03489216789603233, 0.06350446492433548, 0.0360037162899971, 0.03880435600876808, 0.0589592307806015, -0.0789710283279419, -0.032882459461688995, -0.045789338648319244, -0.021030493080615997, -0.05721370130777359, -0.01570642925798893, 0.06773950159549713, 0.0477975532412529, 0.02080758847296238, -0.07664106041193008, 0.04821384325623512, -0.052367933094501495, 0.07436149567365646, -0.024946363642811775, -0.03843500837683678, -0.05065334215760231, -0.008651865646243095, 0.016191929578781128, -0.05380123108625412, 0.04309113323688507, 0.0409851111471653, -0.01066699717193842, 0.021276379004120827, 0.06583339720964432, -0.05280669033527374, -0.01774919219315052, -0.061985645443201065, -0.02252737060189247, -0.012123598717153072, -0.011422254145145416, -0.029680127277970314, -0.05887051299214363, 0.0390109

# Conclusion

Here we have demonstrated how we can implement Ingestion and Enrichment pipeline using redis vector DB by using ML Transfrom's SentenceTransformerEmbeddings for generating the embeddings of the text chunks.